# Remote Modeling of FGFR Kinase Inhibitors with Boltz-1 Using dxflow

### Why This Case Study Matters
FGFR kinases (FGFR1-3) are implicated in diverse cancers. Infigratinib (BGJ-398), an ATP-competitive inhibitor, is approved for FGFR2 fusion-driven cholangiocarcinoma and in trials for broader FGFR-addicted cancers. Understanding the structural basis of its activity supports:

- **Precision medicine** — anticipating resistance mutations and optimizing analogs.
- **Biomarker discovery** — linking structure to downstream gene expression.
- **In-silico screening** — evaluating analogs computationally before synthesis.

Boltz-1, a generative diffusion model, predicts protein-ligand complex structures. This notebook demonstrates how to run Boltz-1 within the DiPhyx platform, integrate molecular modeling and transcriptomics, and interpret biological outcomes.

## Pipeline Overview
| Stage                      | Key Tool                          | Output                                         |
|----------------------------|-----------------------------------|------------------------------------------------|
| A. Target & ligand prep    | UniProt, RDKit                    | FGFR1-3 kinase sequences; 3D mol of Infigratinib |
| B. Structure prediction    | Boltz-1                           | PDBs of FGFR–drug complexes + per-model confidence |
| C. Expression signature    | Scanpy + GSEApy (bulk/SC datasets)| Differential gene lists & pathway NES          |
| D. Interpretation          | PyMOL, volcano/heat-maps          | Structure-function narrative & design hypotheses |


> **Compute Recommendations**: Run this notebook on GPU-enabled units on [DiPhyx](https://www.diphyx.com/). Recommended instances include:
> - `g4dn.4xlarge` (16 cores, 64 GB RAM, Tesla T4 GPU)
> - `g4dn.2xlarge` (8 cores, 16 GB RAM, Tesla T4 GPU)
> - `g6.2xlarge` (8 cores, 32 GB RAM, NVIDIA L4 GPU)

## Practical Walk-through

### Prepare Inputs
Fetch FGFR1 kinase domain and generate Infigratinib conformer:

In [9]:
"""Fetch FGFR1 kinase domain (residues 564‑822) and build a 3‑D conformer of
Infigratinib – all in pure Python so you can run inside a notebook."""

import os, requests, textwrap
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem

boltz_input_path = Path("boltz_inputs");
boltz_input_path.mkdir(exist_ok=True)


# ▸  Fetch canonical SMILES for Infigratinib (PubChem CID 50909836) -----
url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/50909836/property/IsomericSMILES/JSON"
smiles = requests.get(url, timeout=30).json()['PropertyTable']['Properties'][0]['IsomericSMILES']
print("SMILES:", smiles[:60], "…")

# Build 3‑D ligand
mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
AllChem.EmbedMolecule(mol, randomSeed=42)
AllChem.UFFOptimizeMolecule(mol)
Chem.MolToMolFile(mol, boltz_input_path / "Infigratinib.mol")
print("Wrote 3D MOL →", boltz_input_path / "Infigratinib.mol")

SMILES: CC(C)(COP(=O)(O)OP(=O)(O)OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C= …
Wrote 3D MOL → boltz_inputs\Infigratinib.mol


#### Generate YAML programmatically

The snippet below downloads the full FGFR1 sequence from UniProt, slices the kinase domain (564–822), and writes a Boltz‑1 YAML in boltz_inputs/. It re‑uses the smiles variable created in the previous cell:

In [10]:
import os, requests, yaml, textwrap
from pathlib import Path

boltz_input_path = Path("boltz_inputs")
boltz_input_path.mkdir(exist_ok=True)

# ▸  Download full FGFR1 sequence (UniProt P11362) ----------------------


uniprots = {
    "FGFR1": "P11362",
    "FGFR2": "P21802",
    "FGFR3": "P22607",
}

seq_dict = {}
for name, uid in uniprots.items():
    url = f"https://www.uniprot.org/uniprot/{uid}.fasta"
    fasta = requests.get(url, timeout=30).text
    full_seq = "".join(l.strip() for l in fasta.splitlines() if not l.startswith(">"))
    kd_seq = full_seq[563:822]  # slice residues 564‑822 (python 0‑based)
    seq_dict[name] = kd_seq
    print(f"{name}: kinase domain length = {len(kd_seq)} aa")

inputs_dict ={}

for name, kd_seq in seq_dict.items():
    yaml_dict = {
        "version": 1,
        "sequences": [
            {"protein": {"id": "A", "sequence": textwrap.fill(kd_seq, 60)}},
            {"ligand":  {"id": "B", "smiles": smiles}},
        ]
    }
    outfile = os.path.join("boltz_inputs", f"{name.lower()}_infig.yaml")
    with open(outfile, "w") as fh:
        yaml.safe_dump(yaml_dict, fh, sort_keys=False)
    inputs_dict[name] = outfile
    print("Wrote", outfile)

FGFR1: kinase domain length = 259 aa
FGFR2: kinase domain length = 258 aa
FGFR3: kinase domain length = 243 aa
Wrote boltz_inputs\fgfr1_infig.yaml
Wrote boltz_inputs\fgfr2_infig.yaml
Wrote boltz_inputs\fgfr3_infig.yaml


### 3.2  Run Boltz‑1 using dxflow
We want to use dxflow library to run Boltz-1 on DiPhyx. The first step is to install dxrflow and make a session using your DiPhyx `email` and `password`. If you don't have a DiPhyx account, you can create one [here](https://www.diphyx.com/).


In [ ]:
# Lets make sure that dxflow is installed 
!pip install dxflow -U

from dxflow.session import Session
# Create a dxflow session with your credentials
session = Session(email='YOUR@EMAIL', password='YOUR_PASSWORD')

  Attempting uninstall: dxflow
    Found existing installation: dxflow 0.1.5
    Uninstalling dxflow-0.1.5:
      Successfully uninstalled dxflow-0.1.5


Now that you are connected to your DiPhyx account, let's see if you have any running compute-units. If you don't have any compute-units yet, you can create one by going to the [DiPhyx dashboard](https://www.diphyx.com/dashboard) and clicking on "Create Compute Unit".


In [ ]:
compute_unit_manager = session .get_compute_manager()
compute_unit_manager.list()


Name        | Status     | IP            | Cluster Type         | CPU | Memory(GB) | Disk                        
------------+------------+---------------+----------------------+-----+------------+-----------------------------
Boltz_Demo  | PREPARING  | 54.211.250.47 | AWS:g6.2xlarge       | 8   | 32         | {boot: 512, volume: 256}    
TEST        | TERMINATED | 54.235.60.203 | AWS:T2 Medium        | 2   | 4          | {boot: 16, volume: 48}      


> **Note:** If there is no compute-unit, you can either create it via the [DiPhyx dashboard](https://diphyx.com/dashboard/user/compute/units)  
> or create it using the `dxflow` SDK.

Now lets select the `Boltz_Demo` compute-unit with the `ip=<ip_address>`, which is a GPU-enabled unit that we will use to run Boltz-1. 

> **Note:** If you are running this notebook on the same machine, you don't need to specify the `ip` address, as the compute-unit is already connected to your notebook (just use `compute_unit_manager.get_unit()`). However, it is a good practice to specify the `ip` address to ensure that the compute-unit is connected to the correct notebook.


In [ ]:
compute_unit = compute_unit_manager.get_unit(name='Boltz_Demo') # or ip_address='<your_compute_unit_ip_address>'
compute_unit.projects.list() # This will show you the list of the project currently available on the compute unit

No data to display


Now lets get the"Boltz-1" flow from the dxflow registery: 

In [5]:
flow_manager = session.get_flow_registery_manager()
# flow_manager.list() # This will show you the list of the flows currently available on the flow manager
boltz1_flow = flow_manager.get_by_name('Boltz-1') # This will get the flow with the name 'boltz_flow'""
boltz1_flow.get_variables() # This will show you the variables of the flow, their description, type and default value

{'INPUT': {'label': 'Input file or directory',
  'hint': "file.yaml/fasta or directory path. Don't remove /volume/",
  'default': '/volume/boltz_inputs/file.yaml',
  'condition': {'key': 'DOCKER_COMPOSE_PROFILE_A', 'value': 'default'},
  'required': True},
 'OUTPUT': {'label': 'Output directory',
  'hint': "Output directory will be at /volume/output. Don't remove /volume/",
  'default': '/volume/boltz_outout/',
  'condition': {'key': 'DOCKER_COMPOSE_PROFILE_A', 'value': 'default'},
  'required': True},
 'RECYCLE_STEPS': {'label': 'Recycle Steps',
  'hint': 'number of recycling iterations',
  'default': '10',
  'condition': {'key': 'DOCKER_COMPOSE_PROFILE_A', 'value': 'default'},
  'required': False},
 'DIFFUSION_SAMPLES': {'label': 'Diffusion Samples',
  'hint': 'The number of poses for the diffusion model',
  'default': '5',
  'condition': {'key': 'DOCKER_COMPOSE_PROFILE_A', 'value': 'default'},
  'required': False},
 'FLAT32_PERCISION': {'label': 'Float Percision',
  'hint': 'Floatin

### Moving the data to the compute-unit
If you are running this notebook on the same compute-unit as Boltz-1, you can skip this step. If you are running it on your local machine, or a differnt compute-unit, you need to move the data to the compute-unit. The code below moves the yamls files `boltz_inputs` directory to the compute-unit.

In [ ]:
compute_unit.get_storage() # This will initilize the storage manager for the compute unit
   # Destination path in the compute unit storage
for name, input_file in inputs_dict.items():
    # Upload each input file to the compute unit storage
    # Note: Make sure the destination path is correct and accessible in your compute unit environment.
    # The dst path will be under /volume/ path and you should not add it in the dst. 
   compute_unit.storage.upload(src=input_file, dst=str(boltz_input_path))
   print(f"Uploaded {input_file} to {boltz_input_path}")



Uploaded boltz_inputs\fgfr1_infig.yaml to boltz_inputs
Uploaded boltz_inputs\fgfr2_infig.yaml to boltz_inputs
Uploaded boltz_inputs\fgfr3_infig.yaml to boltz_inputs


The next step is to create the project. If this is first time creating boltz-1 project on this compute-unit, it will take couple of minutes to download the container image on this machine.

In [13]:
boltz_job_input_path = f"/volume/{boltz_input_path.name}"  # Path to the input YAML files in the compute unit storage
boltz_job_output_path = "/volume/boltz_output"

variables = {
    "INPUT": boltz_job_input_path,  # Path to the input YAML files in the compute unit storage
    "OUTPUT": boltz_job_output_path,   
    "RECYCLE_STEPS": 10,    
    "DIFFUSION_SAMPLES": 8,
    "OTHER_FLAGS":  "--use_msa_server"
}
boltz_project  = compute_unit.projects.create(flow_name='Boltz-1', variables=variables)

    

Now, lets run the projects 

In [14]:
boltz_project.start()
print("Started Boltz-1 project with variables:", variables)


Project is signaled to START successfully
Started Boltz-1 project with variables: {'INPUT': '/volume/boltz_inputs', 'OUTPUT': '/volume/boltz_output', 'RECYCLE_STEPS': 10, 'DIFFUSION_SAMPLES': 8, 'OTHER_FLAGS': '--use_msa_server'}


To monitor the project's progress, you can view the logs in real-time or access the log history using the following commands:

In [15]:
boltz_project.realtime_logs() # or boltz_project.logs(realtime=True)


Container: 4f3a9b71364d
Date                 Message
2025-05-28 21:58:20  Downloading the CCD dictionary to /volume/boltz_cache/ccd.pkl. You may change the cache directory with the --cache flag.
2025-05-28 21:58:21  Downloading the model weights to /volume/boltz_cache/boltz1_conf.ckpt. You may change the cache directory with the --cache flag.
2025-05-28 21:58:30  Processing input data.
2025-05-28 21:58:30  Checking input data.
2025-05-28 21:58:30  Running predictions for 3 structures
2025-05-28 21:58:35  Generating MSA for /volume/boltz_inputs/fgfr1_infig.yaml with 1 protein entities.
2025-05-28 21:58:35  
2025-05-28 21:58:35    0%|          | 0/3 [00:00<?, ?it/s]
2025-05-28 21:58:35  
2025-05-28 21:58:35    0%|          | 0/150 [elapsed: 00:00 remaining: ?]
2025-05-28 21:58:35  
2025-05-28 21:58:35  COMPLETE:   0%|          | 0/150 [elapsed: 00:00 remaining: ?]
2025-05-28 21:58:35  
2025-05-28 21:58:35  SUBMIT:   0%|          | 0/150 [elapsed: 00:00 remaining: ?]
2025-05-28 21:58:36 

## Result visuzalization
You can visualize the results of the Boltz-1 run using `PyMOL` or any other molecular visualization tool (e.g. `ParaView`). The output PDB files will be located in the `/volume/boltz_outputs` directory on your compute unit.